## Instruksi

Jalankan code berikut pada Google Colab dan PyDroid3 (Android Python Application) di Smartphone Android. Bandingkan hasilnya dan tuliskan analisa anda, tuliskan juga spesifikasi smartphone yang anda gunakan :).

In [6]:
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
from annoy import AnnoyIndex
import hnswlib
import faiss
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Contoh dataset kecil untuk testing
# -------------------------------
np.random.seed(42)
n_samples = 10000   # jumlah database vector
d = 128             # dimensi
X = np.random.random((n_samples, d)).astype('float32')

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact NN (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=d)
p_hnsw.init_index(max_elements=n_samples, ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, _ = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], 100)  # nlist=100 sebagai argumen posisi
index_faiss.train(X_scaled.astype('float32'))
index_faiss.add(X_scaled.astype('float32'))
index_faiss.nprobe = 10
dist_faiss, idx_faiss = index_faiss.search(X_scaled.astype('float32'), k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

# -------------------------------
# Tampilkan ringkasan waktu
# -------------------------------
print("\n=== Ringkasan Waktu (detik) ===")
print(f"Exact NN : {time_exact:.3f}")
print(f"Annoy    : {time_annoy:.3f}")
print(f"HNSW     : {time_hnsw:.3f}")
print(f"FAISS    : {time_faiss:.3f}")


Exact NN done in 1.094 s
Annoy done in 0.680 s
HNSW done in 0.664 s
FAISS IVF done in 0.172 s

=== Ringkasan Waktu (detik) ===
Exact NN : 1.094
Annoy    : 0.680
HNSW     : 0.664
FAISS    : 0.172


![](../Dataset/google-colab.png)

![](../Dataset/PyDroid3.jpg)

## Analisis

Tuliskan analisis perbandingan hasil running di Google Colab, Local dan PyDroid3 di sini.

**Spesifikasi Smartphone:**
- Merk/Model: itel S23+/itel S681LN
- Processor: UMs9230T
- RAM: 16
- OS: Android 13

**Perbandingan Waktu Eksekusi:**

| Metode | Google Colab (s) | Local (s) | PyDroid3 (s) |
|--------|------------------|-----------|--------------|
| Exact NN | 4.543 | 1.094 | 1.437 |
| Annoy | 0.707 | 0.680 | 2.347 |
| HNSW | 5.580 | 0.664 | 11.102 |
| FAISS | 0.416 | 0.172 | - |

**Kesimpulan:**

Semua di local berjalan lebih cepat karena spesifikasi berpengaruh besar pada memrosesan data:)
dan di android faiss tidak bisa di install.

Selamat belajar :)